In [ ]:
import time
import timeit
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg

## Intro

Numpy is a numerical python library implementing almost everything you would want to
use from classical numerical linear algebra. It has a sister package that is closely
integrated in many submodules, and here we will cover a subset of both. The plan is to
briefly go over different data types, data layouts (arrays), functions, linear
algebra, and random number generation. There are many other libraries in Python for
doing deep learning which use quite similar APIs. Almost every concept we discuss in
this module would carry over to those frameworks. Among those frameworks, PyTorch is
the most popular deep learning framework in use today, but JAX is a terrific ecosystem
with much tighter coupling to Numpy.

## Data Types

This section covers the core data types in Numpy. These can be scalars or the 
core data types for arrays. We cover

* ints 
* uints 
* floats
* complex
  
We also look at type promotion in the case where different data types interact 
together. Generally the type promotion rules are that we promote lower order
types up to the larger container. So integers will become floats and small floats
will become big floats if that is necessary.

### Integers

Like Python provides several builtin types to represent numerical data, Numpy provides
many of its own types. A key difference is that all of numpy's types are fixed width,
meaning that they can overflow. For example:

In [ ]:
# Create a variable i64 = np.array(1)


We can also ask for unsigned versions of integers, or cast data from one type into
another (dangerous if you aren't positive of domain restrictions). For example:

In [ ]:
# Create a variable u64 = i64.astype(np.uint64)


The types can sometimes lead to surprising results if we aren't careful. Unlike Python's builtin integers, these fixed-width integers can wrap around to values in unexpected ways. Here are some examples:

In [ ]:
# Compare: 

# i64 - 2, u64 - 2
print(f"{}, {}")

# u64 - 2 == 2**64 - 1
print(f"{}")

# u64 - 2 as an int64
print(f"{}")

The signed integer uses its first bit to indicate the sign of the number and its
remaining 63 bits to represent the integer itself. So the range of representable
values are $-2^{63} \leq i_{64} < 2^{63}$. The unsigned bit does not reserve that first
position, so we have $0 \leq u_{64} < 2^{64}$. And as we see, integers can be recast between
types to be interpreted differently.

Because Numpy stores integers in fixed space, we can also overflow them with undefined
behavior. Consider now what happens when we shift the bit left by several positions:

In [ ]:
# Left-shift i64 by 64 (i64 << 63)


And although we will practically never need to *do* any of this in applications, it is
important to at least know these things could be issues if we're not careful. And it is 
especially important to know this behavior is different from what we get from Python's 
builtin types. Integers in Python are not necessarily the same as integers in Numpy.

### Floats

Floating point types are a similar story. By default, Numpy stores everything in
float64. It also aggressively promotes things internally to float64. So if we just
create a float


In [ ]:
# Define: f64 = np.array(1.0)


# Print f64 and its dtype (f64.dtype)
print()

we see it is using 64 bits. We could just as well store things in float32, in which
case we have

In [ ]:
# Let a = 1.0 and b = 1.0 in float32


# c = a + b


# d = a * b

# Print the values a, b, c, d
print()

# print the dtypes of a, b, c, d
print()

Numpy also supports complex numbers. Unless you're doing signal processing you
probably won't find much need for these. But some of you may want to use FFTs in your
research, in which case you will want to know that these exist and behave slightly
differently from other data types.

In [ ]:
# Create c64 = 1.0 + 1.0j


# Print the value


For example, the absolute value of a complex number is actually the Euclidean distance in the
complex plane:

In [ ]:
# Compute |c64|, the modulus of the complex variable


# Verify this is equal to sqrt(2)

The full list of numpy's dtypes can be found at

   [Numpy Dtypes](https://numpy.org/devdocs/user/basics.types.html)


A last note: floats are imprecise approximations of the real number line. So we have

In [ ]:
# 0.1 + 0.2 in np.float64


# Compare with standard python type


This is a consequence of implementing floats using IEEE754, which is the most common
computational model of the real numbers. The main consequence of floating point arithmetic
is that we cannot always guarantee an operation is going to satisfy `equality` comparisons
as we might expect. Integers do not suffer this affliction, as the next example shows:

In [ ]:
# Let i64 = 1 and u64 = 1 (using the appropriate types)

# Compare the integers with i64 == u64


But floats are not the same story

In [ ]:
# a = 0.1, b = 0.2, c = 0.3

# Compare: a + b == c
print()

# Compare: np.isclose(a + b, c)
print()

In floating point, we have to instead ask *are these two things close*?

## Data Layouts (Arrays)

Topics:

- scalars
- vectors
- matrices
- tensors
- reshaping
- broadcasting

The reason we use Numpy isn't really to work on scalars. It's because we want to
use container types like n-dimensional arrays. We use numpy when we want to do matrix
math, or just to make some analysis in Python go faster than default Python.

A good starter is to look at the arange object. Python has `range(start, stop, step)`
objects. Numpy has `arange(start, stop, step)` objects. Note in both cases that we
use half open intervals [start, stop).

In [ ]:
# Create an arange equivalent to np.arange(0, 10, 1)



These arrays are zero-indexed. The first element is accessed at arange[0], like:

In [ ]:
# Index into the first element of `arange`


Unlike Python's range object, our numpy arange supports instantiation by float. So the
following are equivalent:

In [ ]:
# Initialize `aranged = np.arange(10)` using floats
aranged = 

# Use type coercion to do the same (showing we can elide this step)
arange_double = 

# Compare the two values

We compared the last two arrays by printing and looking at them. But we can actually
use comparison operators for the same purpose:

In [ ]:
# Compare:

# aranged < arange_double
print()

In [ ]:
# aranged == arange_double
print()

In [ ]:
# aranged > arange_double
print()

What happened here? Numpy does comparisons elementwise by _broadcasting_. We will
discuss broadcasting more momentarily. If we wanted to know if all of the elements
were the same, we could wrap the last comparison like so

In [ ]:
# Now get a single bool using `np.all` for aranged == arange_double:
np.all()

Or if we wondered if any of the elements matched, we could instead ask

In [ ]:
# We could have used `np.any` instead
np.any()

These comparisons may take some practice to get used to, but they become intuitive
over time (I promise!).

Another thing to be cognizant of is type promotion. Numpy _really likes to work in
float64_. And any time we combine data from different types, Numpy will promote to the
largest type necessary to store the results.

In [ ]:
arangef = np.arange(10.0, dtype=np.float32)
aranged = np.arange(10.0, dtype=np.float64)

# What is the type of arangef + arangef?
print(().dtype)

# What is the type of arangef + aranged?
print(().dtype)

A last thing we want to discuss before looking at larger array types is broadcasting.
We saw with comparisons that numpy does elementwise comparison. The same is true if we
compare our scalars with our vectors.

In [ ]:
# Create i64 = 5
i64 = np.array()

# Create arange = np.arange(10)
arange = np.arange()

# Compare arange == i64


In [ ]:
# What is np.all() give?
np.all(arange == i64)

In [ ]:
# What does np.any give?
np.any(arange == i64)

Or if we want to know where two things are equal, we can just ask:

In [ ]:
# np.where:
np.where()

This returns an index of all of the places where two things are equal. Often, however
we want to know this so that we can replace some set of values with another set of
values. For example, we typically use `np.where` to replace writing a code snippet like

```python
for i in range(10):
    if arange[i] == i64:    # conditional
        arange[i] = 1       # true vals
    else:
        arange[i] = 0       # false vals
```

And doing that in `np.where`, we have 

```python
np.where(conditional, true_vals, false_vals)
```

In [ ]:
# Compute np.where(cond, true_vals, false_vals)
# NOTE: Do not overwrite the buffer, just print


### Broadcasting

One of the great features of numpy is its declarative syntax, which includes
broadcasting of results when the shapes don't match but have compatible dimensions
somehow. Here are a few examples to try:

In [ ]:
# Define
arange = np.arange()

# Broadcasting addition
arange + 5

In [ ]:
# Multiplication
arange * 5

In [ ]:
# Division
arange / 5

In [ ]:
# Integer division
arange // 5

### Reshaping arrays

Another wonderful feature of numpy arrays is that the shapes
are *mutable*. We can reshape things in any way we like to get
a more meaningful representation of our data:

In [ ]:
# Create a 2x5 array 
arange = np.arange().reshape()
arange

Above we provided both dimensions explicitly, but we can also provide them
implicitly, like so:

In [ ]:
# Repeat, but only specify one dim
arange = np.arange().reshape()
arange

We can make the arrays longer and reshape them into any configuration we like:

In [ ]:
# Make a tensor with shape (2, 3, 5, 7)
arange = np.arange().reshape()
arange

### Array Metadata

Arrays come with metadata: We can query the amount of storage used for the array, its
shape, and the dtype (as we saw with scalars):

In [ ]:
# Print the following:

# nbytes

# shape 

# dtype

Data can be reshaped in whatever way we want, and again we can query the new arrays
to learn what the shape and data types are. In most cases it is probably clear from
context what we want to use or do

In [ ]:
# Reshape: arange (2, 3, 5, 7) -> (5, 2, 3, 7)
arange_reshaped = 

# Print nbytes

# Print shape 

# Print dtype

# Print data


### Array Slicing

Finally, we can take subsets of data in the same way as we slice any other object in
Python; some examples:

In [ ]:
arange = np.arange(10.0)

# slice elements 3:6 of arange


In [ ]:
# Slice the first four elements of arange


In [ ]:
# Slice from index 7 to the end of the array


In [ ]:
# Slice the last four elements of arange


### Initializing Data

There are several ways in which we may want to create new arrays aside from the ones
shown. Some of these are with pseudo-random numbers, which we'll get to in a moment.
But there are several deterministic sequences we may want to create, like

In [ ]:
# Generate a 3x3 empty matrix


In [ ]:
# Generate a 3x3 zeros matrix


In [ ]:
# Generate a 3x3 ones matrix


Here are a few more variations to try:

In [ ]:
# Generate a sequence 0, 2, 4, ..., 30 using the arange function
# Note its signature: np.arange(start, stop, step)


In [ ]:
# Generate a sequence 0.00, 0.01, ..., 0.98, 0.99, 1.00
# Recall np.arange(start, stop, step) allows floats!


In [ ]:
# Generate the same sequence [0.00, 0.01, ..., 0.99, 1.00] using the
# np.linspace(start, stop, num) function


We can also index into arrays by conditional statements, as follows
(note this is related to the `np.where` functions from before):

In [ ]:
# Query: arange > 4


In [ ]:
# Query: not (arange > 4) using ~


### Stacking data

The last major thing to do with arrays is to stack them in various ways.
There are several functions for doing this. I personally like to stick
to `np.concatenate` when I want to extend one array with another, and 
`np.stack` if I want to do anything else.

In [ ]:
x = np.linspace(0, 1, 20)

# Concatenate x with x
concat = np.concatenate(...)
print(concat.shape)

concat

In [ ]:
# Vertically stack
vstack = np.stack(..., axis=...)

# Print the shape and values
print(vstack.shape)
vstack

In [ ]:
# Horizontally stack 
hstack = np.stack(..., axis=...)
print(hstack.shape)
hstack

Or use these, but it's simpler to remember one function than it is to remember two or
three, so I prefer stack with an argument about how I want to stack things.

```python
vstack = np.vstack([x, x])
hstack = np.hstack([x[:, None], x[:, None]])
```

## Random Numbers

We present this module next because it is imperative to generating data in the
subsequent sections, so that we can work on less contrived examples than the ones we
have already seen.

### Random Seeds

We don't have to set random seeds, but it makes our analysis reproducible if we
implement something and want it to run deterministically. Everything we explore today
will be seeded. 

The right practice is to set seeds once at the start of a program and
then execute the program sequentially from start to finish. You will notice if you run
cells multiple times after setting the seed that the results become apparently random.
This is one of the primary reasons to strive for running stuff in scripts rather than
jupyter notebooks. That is a point that will be difficult to get across by the end of
the workshop, but it is one to consider if you continue to work in Python after this
workshop. For today, however, we are going to work in Jupyter notebooks because they
are interactive and nice for exploratory work.

The seed itself could be lots of things, including an empty argument. I always start
with a concrete value... Most of my test scripts start with seed 0. If you want to
have random but reproducible behavior, a better seed is to just grab the system time
in nanoseconds at program startup, like this:

In [ ]:
# Get time in nanoseconds


# Print the seed (decimal and hex)
print(f"{seed = }, (in hex: 0x{seed:x})")

# Create a Random Number Generator (rng) object with the seed
rng = 

**But don't forget to save it when you're done!**


If we save that seed someplace then we can always reconstruct the program's execution
using the seeded value. This is *immensely* helpful for debugging programs that rely
on random number generation!

The random number generator supports generating all kinds of different distributions.
Here are some of them:

In [ ]:
# Make three subplots with shape (1, 3), a large enough figsize to plot, and constrained layout
fig, ax = plt.subplots(1, 3, figsize=(12, 3), layout="constrained")

# Generate 500 normal rvs and plot in ax[0]
normal_rvs = 

ax[0].hist(normal_rvs, density=True, edgecolor="C0", facecolor="lightblue")
ax[0].set_title("X ~ N(0, 1)")

# Generate 500 uniform rvs and plot in ax[1]
uniform_rvs = 

ax[1].hist(uniform_rvs, density=True, edgecolor="C0", facecolor="lightblue")
ax[1].set_title("Y ~ U(0, 1)")

# Generate 500 gamma rvs with shape 5.0 and scale 3.0 and plot in ax[2]
gamma_rvs = 

ax[2].hist(gamma_rvs, density=True, edgecolor="C0", facecolor="lightblue")
ax[2].set_title("Z ~ Gamma(5.0, 3.0)")

# Show the results
plt.show()

There are also great helpers for randomly permuting data, like so:

In [ ]:
# permute `normal_rvs[:10]`
normal_rvs_perm = rng.permutation(..., axis=)

# Compare the two vectors
print(normal_rvs[:10])
print(normal_rvs_perm[:10])

If we wanted to permute *all* of our data by the same index, we could also
generate the permutation over the index 0..500:

In [ ]:
# Generate a permutation array of 20 elements
permutation = rng.permutation()
permutation

Now we could reindex all three vectors by this index. This is great for data splitting in several machine learning contexts.

A similar choice for doing the same thing is to use `rng.choice` (`np.random.choice`). The benefit of `rng.choice` is that it 
allows you to set the probability of drawing each datum and it allows repeated draws of the same data. Bootstrapped confidence
are an example of where this is good to know.

In [ ]:
# Generate a sample using rng.choice
sample_idx = rng.choice(20, size=10, replace=False)
print(sample_idx)

As with the permutations, we can also just shuffle data in this way directly:

In [ ]:
# Resample the data using rng.choice
normal_rvs_sample = rng.choice(normal_rvs[:20], size=30, replace=True)

plt.plot(normal_rvs[:20], ".")
plt.plot(normal_rvs_sample, ".")
plt.show()

And if what we want to do is genuinely randomizing the order of data, we can
call `rng.shuffle` (`np.random.shuffle`):

In [ ]:
# Simulate a deck of cards:
deck = 

# Shuffle the deck (done in place rather than returning a copy):
rng.shuffle(...)

# Print
deck

## Functions

We have already seen several functions. This is a short tour of the more obvious ones that we
didn't cover yet.

### Reductions and Summary Statistics

Since we just looked at random data, let's start the overview of functions by looking at 
summary statistics.

In [ ]:
# Generate X ~ N(0, 1) with size (100, 30, 5)
x = 

To start, some functions are bound to the variables we are working with. For example,
we can calculate the mean, variance (var), standard deviation (std), minimum (min),
maximum (max), index of the min (argmin), and index of the max (argmax) using those
functions bound to the numpy namespace _or_ bound to the array objects themselves.

In [ ]:
# Compare: np.mean(x) and x.mean()


In [ ]:
# Compare: np.var(x) and x.var()


In [ ]:
# Compare: np.std(x) and x.std()


In [ ]:
# Compare: np.min(x) and x.min()


In [ ]:
# Compare: np.max(x) and x.max()


In [ ]:
# argmin


In [ ]:
# argmax


We can also apply these functions over specific axes. This operation will reduce the
shape of the array along that dimension, which is avoidable if we use `keepdims=True`.
Often that is unnecessary.

In [ ]:
# Calculate summary stats along different axes and note the shapes/values
print(f"Calculating means for {x.shape=} along different axes")

# For axis dim 0, 1, 2, compute and print:
# np.mean(..., axis= ).shape




### Other functions

But statistics aren't the only functions we may want to compute. Here is a plot of
sines, cosines, polynomials, and some of the other things you may generally want.

In [ ]:
# define x on (0, 8] with 100 valus
x = np.linspace(1e-16, 8.0, 100)

# Make a 2,4 array of subplots:
fig, axes = plt.subplots(2, 4, figsize=(8, 4), layout="constrained", sharex=True)
ax = axes.flatten()

# sin(x)
ax[0].plot()
ax[0].set_title("sin(x)")

# cos(x)
ax[1].plot()
ax[1].set_title("cos(x)")

# tan(x)
ax[2].plot()
ax[2].set_title("tan(x)")

# square(x)
ax[3].plot()
ax[3].set_title("$x^2$")

# cube(x)
ax[4].plot()
ax[4].set_title("$x^3$")

# sqrt(x)
ax[5].plot()
ax[5].set_title("$\\sqrt{x}$")

# log(x)
ax[6].plot()
ax[6].set_title("log(x)")

# exp(x)
ax[7].plot()
ax[7].set_title("exp(x)")

fig.suptitle("A Sample of Numpy functions to calculate")
plt.show()


And of course, all of the other operations you'd expect to work do work. They operate element-wise on data:

In [ ]:
# Generate some small x (~20)
x = 

# Define y = sin(x)
y = 

# View
np.sqrt(x) + 5 * y**2 / 3 + 2

## Linear Algebra

And now for the thing Numpy really shines at (other than random number generation), linear algebra!

We can warm up with the dot product:
$$
    \langle u, v \rangle = u \cdot v = \sum_{k=1}^{n} u_i v_i.
$$

In [ ]:
# Generate u, v ~ N(0, 1) of size 30
u, v = rng.normal(size=(..., ...))

# Compute the dot product between u and v
np.dot(u, v)

# NOTE: We can compute this as u.dot(v) also

Next we can solve linear systems of the form $A x = b$. Consider the system
$$
    \begin{bmatrix}
        0 & 1 & 2 \\
        1 & 4 & 2 \\
        2 & 1 & 1 \\
    \end{bmatrix}
    x = 
    \begin{bmatrix}
        8 \\ 15 \\ 7
    \end{bmatrix}.
$$
We'd like to solve this for $x$. The function `np.linalg.solve` does this for us straight away using an LU solve
under the hood.

In [ ]:
A = np.array(
    [
        [0, 1, 2],
        [1, 4, 2],
        [2, 1, 1],
    ],
)

x = np.array([1, 2, 3])
b = np.dot(A, x)

A, x, b

In [ ]:
# Solve for x_solved using np.linalg.solve
x_solved = 

# Calc norm of the difference x_solved - x
norm = np.linalg.norm(___ - ___)

# Print the norm
print(f"||x_solved - x|| = {_____:.4f}")

Some of you will have problems where the matrix has special structure (whether
you are initially aware of this or not). When such structure does exist, you can often
exploit it in the solver you are using. For example, here is a circulant embeddings
matrix. 

In [ ]:
# Generate a sequence of length 500
c = np.arange(...)

# Use linalg.circulant(c)
A = 

# Generate X ~ N(0, 1) multiplying with A


# Define b = A x


Now let's compare the time it takes to solve these using the generalized `np.linalg.solve` versus Scipy's 
specialized `scipy.linalg.solve_circulant`. 

In [ ]:
%%timeit

# Solve with np.linalg.solve
linalg.solve(A, b)

In [ ]:
%%timeit 

# Solve with linalg.solve_circulant. 
# NOTE: solve_circulant uses the first column of A only with signature
# linalg.solve_circulant(c, b)
linalg.solve_circulant(A[:, 0], b)

That's roughly a 10x speedup!

### Matrix Decompositions

There are several decompositions that are also supported in np.linalg and scipy.linalg
(imported as `from scipy import linalg`). Some of these that are common to least
squares problems are shown in `./ols.py`. Others we don't cover are the SVD and
eigen decompositions. These would be most relevant to PCA. However, good
implementations of these are typically provided in scikit-learn also via
`sklearn.decomposition`. Consider having a look at those if you are planning to do PCA
in your research.